In [3]:
import pandas as pd
import numpy as numpy
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
data = pd.read_csv(r"E:\cursor\IPO chart preparation and performance measure\IPO_data.csv")

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [7]:
data = data.dropna(subset=['Is_Successful'])

features = ['Issue Price', 'Listing Day Close', 'Listing Day Gain', 'Current Price']
df = data.dropna(subset=features)  

X = df[features]
y = df['Is_Successful']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [8]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_rf_pred = rf.predict(X_test)

print("Random Forest Accuracy:", accuracy_score(y_test, y_rf_pred))
print(classification_report(y_test, y_rf_pred))


Random Forest Accuracy: 0.9464285714285714
              precision    recall  f1-score   support

           0       0.95      1.00      0.97        52
           1       1.00      0.25      0.40         4

    accuracy                           0.95        56
   macro avg       0.97      0.62      0.69        56
weighted avg       0.95      0.95      0.93        56



In [9]:
import joblib
joblib.dump(rf, "rf_model.pkl")
joblib.dump(scaler, "scaler.pkl")

['scaler.pkl']

In [12]:
import gradio as gr
import numpy as np
import joblib

# Load model and scaler
model = joblib.load("rf_model.pkl")
scaler = joblib.load("scaler.pkl")

# Ideal benchmarks from your data
IDEAL = {
    "Issue Price": (45.5, 52),
    "Listing Day Gain (%)": (2.5, 31.67),
    "Current Price": 1110.43,
    "Profit/Loss (%)": 20.67,
}

# Generate insights based on benchmarks
def generate_insights(issue_price, gain_percent, current_price):
    insights = []

    # Listing Day Gain
    if gain_percent >= IDEAL["Listing Day Gain (%)"][0]:
        insights.append(f"✅ Listing Gain: {gain_percent:.2f}% ≥ {IDEAL['Listing Day Gain (%)'][0]}%")
    else:
        insights.append(f"⚠️ Listing Gain too low: {gain_percent:.2f}% < {IDEAL['Listing Day Gain (%)'][0]}%")

    # Current Price Momentum
    ideal_cp_threshold = issue_price * 1.10
    if current_price >= ideal_cp_threshold:
        insights.append(f"✅ Current Price ₹{current_price:.2f} ≥ ₹{ideal_cp_threshold:.2f} (10% above issue)")
    else:
        insights.append(f"⚠️ Current Price ₹{current_price:.2f} < ₹{ideal_cp_threshold:.2f} — Weak momentum")

    # Issue Price Range
    if IDEAL["Issue Price"][0] <= issue_price <= IDEAL["Issue Price"][1]:
        insights.append(f"✅ Issue Price ₹{issue_price:.2f} — In ideal range (₹{IDEAL['Issue Price'][0]}–₹{IDEAL['Issue Price'][1]})")
    else:
        insights.append(f"⚠️ Issue Price ₹{issue_price:.2f} — Outside ideal range (₹{IDEAL['Issue Price'][0]}–₹{IDEAL['Issue Price'][1]})")

    return "\n".join(insights)

# Prediction and benchmark comparison
def predict_and_explain(issue_price, gain_percent, current_price):
    try:
        gain = gain_percent / 100.0
        listing_close = issue_price * (1 + gain)
        profit_loss = ((current_price - issue_price) / issue_price) * 100

        features = np.array([[issue_price, listing_close, gain, current_price]])
        scaled = scaler.transform(features)
        prediction = model.predict(scaled)[0]
        proba = model.predict_proba(scaled)[0][1]

        label = "✅ Successful IPO!" if prediction == 1 else "❌ Unsuccessful IPO"
        label += f" (Confidence: {proba:.2f})"

        # Insights
        insights = generate_insights(issue_price, gain_percent, current_price)
        listing_close_str = f"📈 Listing Close = ₹{listing_close:.2f}\n💼 Profit/Loss = {profit_loss:.2f}%\n"
        return label, listing_close_str + insights

    except Exception as e:
        return "⚠️ Error", f"Error in processing: {e}"

# Gradio interface
iface = gr.Interface(
    fn=predict_and_explain,
    inputs=[
        gr.Number(label="Issue Price (₹)"),
        gr.Number(label="Listing Day Gain (%)"),
        gr.Number(label="Current Price (₹)")
    ],
    outputs=[
        gr.Text(label="IPO Prediction & Confidence"),
        gr.Text(label="Numerical Insights + Benchmarks")
    ],
    title="📊 IPO Success Predictor with Real Benchmarks",
    description="Enter your IPO parameters and compare with benchmark values from past successful IPOs."
)

iface.launch()


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


C:\Users\Aneesh Mada\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Aneesh Mada\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Aneesh Mada\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Aneesh Mada\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\Aneesh Mada\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Us